In [2]:
import IPython.display as ipd
import librosa as lr
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

import utilty

# %config InlineBackend.figure_format = 'svg'
plt.rcParams['figure.figsize'] = (8, 3)
ipd.display(ipd.HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
track = utilty.get_audio_tracks()[0]
track = utilty.find_file('without')

sec = 30
duration = utilty.get_duration(track)
segments = int(duration) // sec
audio_30secs = [lr.load(track, offset=sec*i, duration=sec) for i in range(segments)]
name = track.stem.split('-')[-1].replace('_', ' ').title()
track

PosixPath('/Users/matteding/Desktop/Whabam/tracks/eminem-2002-the_eminem_show-10-without_me.mp3')

In [4]:
# all features with shape (1, t)
feature_funcs = [lr.feature.rmse, lr.feature.spectral_bandwidth, lr.feature.spectral_centroid, 
                 lr.feature.spectral_flatness, lr.feature.spectral_rolloff, lr.feature.zero_crossing_rate]
aggs = []
for audio, _ in audio_30secs:
    segment = dict()
    for func in feature_funcs:
        result = func(audio)
        agg = utilty.get_aggregations(result)
        segment[func.__name__] = agg
    aggs.append(segment)

In [5]:
aggs[0]['spectral_flatness']

Aggregations(mean=0.022005646, median=0.0037256456, mode=ModeResult(mode=array([9.538462e-08], dtype=float32), count=array([1])), max=0.21852238, min=9.538462e-08, std=0.038795024, sum=28.431293)

In [6]:
tonnetzes = [lr.feature.tonnetz(audio) for audio, _ in audio_30secs]
tonnetz_aggs = [utilty.get_aggregations(tonnetz, axis=1) for tonnetz in tonnetzes]
tonnetz_map = {0: 'x_P5', 1: 'y_P5', 2: 'x_m3', 3: 'y_m3', 4: 'x_M3', 5: 'y_M3'}

In [7]:
tonnetz_aggs[0].mean.shape # 6 for perfect 5ths, major 3rds, minor 3rds with x and y

(6,)

In [9]:
tnz = tonnetz_aggs[0]

In [14]:
tnz._fields

('mean', 'median', 'mode', 'max', 'min', 'std', 'sum')

In [17]:
for field, agg in tnz._asdict().items():
    print(field)
    print('\t', agg)

mean
	 [ 0.01082346  0.01750369  0.13759612 -0.04362405  0.01900487 -0.01276979]
median
	 [ 0.01053545  0.01768655  0.13372641 -0.02555879  0.01944662 -0.01574702]
mode
	 ModeResult(mode=array([[-0.08937595],
       [-0.06368575],
       [-0.26841515],
       [-0.42429943],
       [-0.08670658],
       [-0.09132711]]), count=array([[1],
       [1],
       [1],
       [1],
       [1],
       [1]]))
max
	 [0.12209626 0.09531961 0.44775712 0.236717   0.101992   0.10537972]
min
	 [-0.08937595 -0.06368575 -0.26841515 -0.42429943 -0.08670658 -0.09132711]
std
	 [0.02869707 0.02211437 0.13196933 0.09281758 0.03923335 0.02852125]
sum
	 [ 13.98390864  22.61476581 177.77418751 -56.36227596  24.55429309
 -16.49857241]


In [24]:
tuple(tnz._asdict().values())

(array([ 0.01082346,  0.01750369,  0.13759612, -0.04362405,  0.01900487,
        -0.01276979]),
 array([ 0.01053545,  0.01768655,  0.13372641, -0.02555879,  0.01944662,
        -0.01574702]),
 ModeResult(mode=array([[-0.08937595],
        [-0.06368575],
        [-0.26841515],
        [-0.42429943],
        [-0.08670658],
        [-0.09132711]]), count=array([[1],
        [1],
        [1],
        [1],
        [1],
        [1]])),
 array([0.12209626, 0.09531961, 0.44775712, 0.236717  , 0.101992  ,
        0.10537972]),
 array([-0.08937595, -0.06368575, -0.26841515, -0.42429943, -0.08670658,
        -0.09132711]),
 array([0.02869707, 0.02211437, 0.13196933, 0.09281758, 0.03923335,
        0.02852125]),
 array([ 13.98390864,  22.61476581, 177.77418751, -56.36227596,
         24.55429309, -16.49857241]))

In [27]:
def tonnetz_aggs(audio_segment):
    """Return tonnetz_aggs from audio_segment"""
    tonnetz = lr.feature.tonnetz(audio)
    tonnetz_aggs = utilty.get_aggregations(tonnetz, axis=1)
    tonnetz_ids = 'x_P5 y_P5 x_m3 y_m3 x_M3 y_M3'.split()
    tonnetz_dct = {}
    for field, agg_arr in tonnetz_aggs._asdict().items():
        
    
    return {id: agg for id, agg in zip(tonnetz_ids, tonnetz_aggs._asdict().values())}

In [29]:
tonnetz_aggs(audio)

{'x_P5': array([ 0.01428755,  0.01690418,  0.14570838, -0.0501279 ,  0.02151135,
        -0.01199497]),
 'y_P5': array([ 0.0138555 ,  0.01652153,  0.14819762, -0.02714751,  0.02441909,
        -0.01689105]),
 'x_m3': ModeResult(mode=array([[-0.09310386],
        [-0.05089711],
        [-0.17309714],
        [-0.40716162],
        [-0.08455134],
        [-0.08348381]]), count=array([[1],
        [1],
        [1],
        [1],
        [1],
        [1]])),
 'y_m3': array([0.13017412, 0.10159274, 0.4034588 , 0.09436358, 0.10272051,
        0.09731782]),
 'x_M3': array([-0.09310386, -0.05089711, -0.17309714, -0.40716162, -0.08455134,
        -0.08348381]),
 'y_M3': array([0.02753574, 0.02159649, 0.12742599, 0.09356664, 0.04020301,
        0.02847409])}

In [ ]:
from importlib import reload
reload(utilty);